In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1357837,2021-06-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1357838,2021-06-11,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1357839,2021-06-12,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1357840,2021-06-13,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
35983,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
35985,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
35987,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
35989,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
35991,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
832826,2021-06-10,Arkansas,Arkansas,5001,2148,37.00,5000,Arkansas,AR,Arkansas,State,3017804
832828,2021-06-11,Arkansas,Arkansas,5001,2148,38.00,5000,Arkansas,AR,Arkansas,State,3017804
832830,2021-06-12,Arkansas,Arkansas,5001,2157,38.00,5000,Arkansas,AR,Arkansas,State,3017804
832832,2021-06-13,Arkansas,Arkansas,5001,2157,38.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1357837,2021-06-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1357838,2021-06-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1357839,2021-06-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1357840,2021-06-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-06-14') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
510,2021-06-14,Snohomish,Washington,39584,586.00,53061,WA,County,822083,4815.09,71.28
1018,2021-06-14,Cook,Illinois,555902,10982.00,17031,IL,County,5150233,10793.73,213.23
1525,2021-06-14,Orange,California,272745,5103.00,6059,CA,County,3175692,8588.52,160.69
2031,2021-06-14,Maricopa,Arizona,554925,10157.00,4013,AZ,County,4485414,12371.77,226.45
2537,2021-06-14,Los Angeles,California,1247103,24433.00,6037,CA,County,10039107,12422.45,243.38
...,...,...,...,...,...,...,...,...,...,...,...
1356985,2021-06-14,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08
1357230,2021-06-14,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1357444,2021-06-14,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1357654,2021-06-14,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
510,2021-06-14,Snohomish,Washington,39584,586.00,53061,WA,County,822083,4815.09,71.28,71.28,4815.09
1018,2021-06-14,Cook,Illinois,555902,10982.00,17031,IL,County,5150233,10793.73,213.23,213.23,10793.73
1525,2021-06-14,Orange,California,272745,5103.00,6059,CA,County,3175692,8588.52,160.69,160.69,8588.52
2031,2021-06-14,Maricopa,Arizona,554925,10157.00,4013,AZ,County,4485414,12371.77,226.45,226.45,12371.77
2537,2021-06-14,Los Angeles,California,1247103,24433.00,6037,CA,County,10039107,12422.45,243.38,243.38,12422.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356985,2021-06-14,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08,75.08,2627.63
1357230,2021-06-14,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1357444,2021-06-14,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1357654,2021-06-14,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
870929,2021-06-14,Hale,Alabama,2266,78.00,1065,AL,County,14651,15466.52,532.39,532.39,15466.52,1
948199,2021-06-14,Clarke,Alabama,3528,61.00,1025,AL,County,23622,14935.23,258.23,258.23,14935.23,2
681081,2021-06-14,Lowndes,Alabama,1420,53.00,1085,AL,County,9726,14600.04,544.93,544.93,14600.04,3
603756,2021-06-14,Etowah,Alabama,14160,362.00,1055,AL,County,102268,13845.97,353.97,353.97,13845.97,4
506466,2021-06-14,Franklin,Alabama,4307,83.00,1059,AL,County,31362,13733.18,264.65,264.65,13733.18,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285089,2021-06-14,Platte,Wyoming,670,11.00,56031,WY,County,8393,7982.84,131.06,131.06,7982.84,19
831061,2021-06-14,Converse,Wyoming,1029,17.00,56009,WY,County,13822,7444.65,122.99,122.99,7444.65,20
1033806,2021-06-14,Lincoln,Wyoming,1415,12.00,56023,WY,County,19830,7135.65,60.51,60.51,7135.65,21
1151072,2021-06-14,Niobrara,Wyoming,162,2.00,56027,WY,County,2356,6876.06,84.89,84.89,6876.06,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
681081,2021-06-14,Lowndes,Alabama,1420,53.00,1085,AL,County,9726,14600.04,544.93,544.93,14600.04,3,1
870929,2021-06-14,Hale,Alabama,2266,78.00,1065,AL,County,14651,15466.52,532.39,532.39,15466.52,1,2
601968,2021-06-14,Clay,Alabama,1601,59.00,1027,AL,County,13235,12096.71,445.79,445.79,12096.71,20,3
289255,2021-06-14,Walker,Alabama,7330,280.00,1127,AL,County,63521,11539.49,440.80,440.80,11539.49,28,4
603309,2021-06-14,Dallas,Alabama,3606,156.00,1047,AL,County,37196,9694.59,419.40,419.40,9694.59,55,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732817,2021-06-14,Hot Springs,Wyoming,371,3.00,56017,WY,County,4413,8406.98,67.98,67.98,8406.98,17,19
1007964,2021-06-14,Uinta,Wyoming,2301,13.00,56041,WY,County,20226,11376.45,64.27,64.27,11376.45,5,20
1033806,2021-06-14,Lincoln,Wyoming,1415,12.00,56023,WY,County,19830,7135.65,60.51,60.51,7135.65,21,21
287896,2021-06-14,Teton,Wyoming,3795,11.00,56039,WY,County,23464,16173.71,46.88,46.88,16173.71,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.28,4815.09,20,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.28,4815.09,20,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.28,4815.09,20,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.28,4815.09,20,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.28,4815.09,20,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1354666,2021-06-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1354667,2021-06-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1354668,2021-06-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1354669,2021-06-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
867317,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15466.52,2,1,1.00
867318,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15466.52,2,1,0.00
867319,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15466.52,2,1,0.00
867320,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15466.52,2,1,0.00
867321,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,15466.52,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051170,2021-06-10,Crook,Wyoming,452,12.00,56011,WY,County,7584,5959.92,158.23,158.23,5959.92,6,23,1.00
1051171,2021-06-11,Crook,Wyoming,452,12.00,56011,WY,County,7584,5959.92,158.23,158.23,5959.92,6,23,0.00
1051172,2021-06-12,Crook,Wyoming,452,12.00,56011,WY,County,7584,5959.92,158.23,158.23,5959.92,6,23,0.00
1051173,2021-06-13,Crook,Wyoming,452,12.00,56011,WY,County,7584,5959.92,158.23,158.23,5959.92,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
678353,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14600.04,1,3,1.00,0.00
678354,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14600.04,1,3,0.00,0.00
678355,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14600.04,1,3,0.00,0.00
678356,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14600.04,1,3,0.00,0.00
678357,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14600.04,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676118,2021-06-10,Albany,Wyoming,4558,15.00,56001,WY,County,38880,11723.25,38.58,38.58,11751.54,23,4,3.00,0.00
676119,2021-06-11,Albany,Wyoming,4564,15.00,56001,WY,County,38880,11738.68,38.58,38.58,11751.54,23,4,6.00,0.00
676120,2021-06-12,Albany,Wyoming,4564,15.00,56001,WY,County,38880,11738.68,38.58,38.58,11751.54,23,4,0.00,0.00
676121,2021-06-13,Albany,Wyoming,4564,15.00,56001,WY,County,38880,11738.68,38.58,38.58,11751.54,23,4,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-06-14') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
345395,2021-06-14,Imperial,California,28800,741.00,6025,CA,County,181215,15892.72,408.91,408.91,15892.72,1,2,1.00,1.00
2537,2021-06-14,Los Angeles,California,1247103,24433.00,6037,CA,County,10039107,12422.45,243.38,243.38,12422.45,2,5,129.00,-1.00
171807,2021-06-14,San Bernardino,California,299014,4788.00,6071,CA,County,2180085,13715.70,219.62,219.62,13715.70,3,4,38.00,0.00
605496,2021-06-14,Inyo,California,1434,38.00,6027,CA,County,18039,7949.44,210.65,210.65,7949.44,4,25,1.00,0.00
76741,2021-06-14,Stanislaus,California,63266,1068.00,6099,CA,County,550660,11489.12,193.95,193.95,11489.12,5,9,76.00,0.00
61035,2021-06-14,San Joaquin,California,74419,1432.00,6077,CA,County,762148,9764.38,187.89,187.89,9764.38,6,17,18.00,0.00
35517,2021-06-14,Riverside,California,301329,4620.00,6065,CA,County,2470546,12196.86,187.00,187.00,12196.86,7,7,17.00,2.00
95178,2021-06-14,Tulare,California,49734,849.00,6107,CA,County,466195,10668.07,182.11,182.11,10668.07,8,10,-24.00,0.00
34587,2021-06-14,Fresno,California,102795,1719.00,6019,CA,County,999101,10288.75,172.05,172.05,10288.75,9,14,12.00,-1.00
548617,2021-06-14,Merced,California,32272,474.00,6047,CA,County,277680,11622.01,170.70,170.70,11622.01,10,8,19.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1284640,2021-06-14,Lassen,California,5775,24.00,6035,CA,County,30573,18889.22,78.50,78.50,18889.22,35,1,0.00,0.00
345395,2021-06-14,Imperial,California,28800,741.00,6025,CA,County,181215,15892.72,408.91,408.91,15892.72,1,2,1.00,1.00
734540,2021-06-14,Kings,California,23170,247.00,6031,CA,County,152940,15149.73,161.50,161.50,15149.73,11,3,2.00,0.00
171807,2021-06-14,San Bernardino,California,299014,4788.00,6071,CA,County,2180085,13715.70,219.62,219.62,13715.70,3,4,38.00,0.00
2537,2021-06-14,Los Angeles,California,1247103,24433.00,6037,CA,County,10039107,12422.45,243.38,243.38,12422.45,2,5,129.00,-1.00
216081,2021-06-14,Kern,California,110617,1400.00,6029,CA,County,900202,12288.02,155.52,155.52,12288.02,14,6,13.00,0.00
35517,2021-06-14,Riverside,California,301329,4620.00,6065,CA,County,2470546,12196.86,187.00,187.00,12196.86,7,7,17.00,2.00
548617,2021-06-14,Merced,California,32272,474.00,6047,CA,County,277680,11622.01,170.70,170.70,11622.01,10,8,19.00,0.00
76741,2021-06-14,Stanislaus,California,63266,1068.00,6099,CA,County,550660,11489.12,193.95,193.95,11489.12,5,9,76.00,0.00
95178,2021-06-14,Tulare,California,49734,849.00,6107,CA,County,466195,10668.07,182.11,182.11,10668.07,8,10,-24.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
345395,2021-06-14,Imperial,California,28800,741.00,6025,CA,County,181215,15892.72,408.91,408.91,15892.72,1,2,1.00,1.00
2537,2021-06-14,Los Angeles,California,1247103,24433.00,6037,CA,County,10039107,12422.45,243.38,243.38,12422.45,2,5,129.00,-1.00
171807,2021-06-14,San Bernardino,California,299014,4788.00,6071,CA,County,2180085,13715.70,219.62,219.62,13715.70,3,4,38.00,0.00
605496,2021-06-14,Inyo,California,1434,38.00,6027,CA,County,18039,7949.44,210.65,210.65,7949.44,4,25,1.00,0.00
76741,2021-06-14,Stanislaus,California,63266,1068.00,6099,CA,County,550660,11489.12,193.95,193.95,11489.12,5,9,76.00,0.00
61035,2021-06-14,San Joaquin,California,74419,1432.00,6077,CA,County,762148,9764.38,187.89,187.89,9764.38,6,17,18.00,0.00
35517,2021-06-14,Riverside,California,301329,4620.00,6065,CA,County,2470546,12196.86,187.00,187.00,12196.86,7,7,17.00,2.00
95178,2021-06-14,Tulare,California,49734,849.00,6107,CA,County,466195,10668.07,182.11,182.11,10668.07,8,10,-24.00,0.00
34587,2021-06-14,Fresno,California,102795,1719.00,6019,CA,County,999101,10288.75,172.05,172.05,10288.75,9,14,12.00,-1.00
548617,2021-06-14,Merced,California,32272,474.00,6047,CA,County,277680,11622.01,170.70,170.70,11622.01,10,8,19.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,408.91,15892.72,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,408.91,15892.72,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,408.91,15892.72,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,408.91,15892.72,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,408.91,15892.72,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4532,06/10/21,Lassen,5770,24.00,18872.86,78.50,78.50,18889.22,35,1,2.00,0.00
4533,06/11/21,Lassen,5771,24.00,18876.13,78.50,78.50,18889.22,35,1,1.00,0.00
4534,06/12/21,Lassen,5775,24.00,18889.22,78.50,78.50,18889.22,35,1,4.00,0.00
4535,06/13/21,Lassen,5775,24.00,18889.22,78.50,78.50,18889.22,35,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,408.91,15892.72,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,408.91,15892.72,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,408.91,15892.72,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,408.91,15892.72,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,408.91,15892.72,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4618,06/10/21,Merced,32241,474.00,11610.85,170.70,170.70,11622.01,10,8,7.00,0.00
4619,06/11/21,Merced,32253,474.00,11615.17,170.70,170.70,11622.01,10,8,12.00,0.00
4620,06/12/21,Merced,32253,474.00,11615.17,170.70,170.70,11622.01,10,8,0.00,0.00
4621,06/13/21,Merced,32253,474.00,11615.17,170.70,170.70,11622.01,10,8,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)